In [1]:
import pandas as pd
import numpy as np
import pickle
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge



In [2]:
# Load the dataset and preprocess it
df = pd.read_csv('jamboree_admission.csv').drop(columns=['Serial No.'])

# Clean and preprocess the data
df.rename(columns={'Chance of Admit ': 'Chance_of_Admit', 'LOR ': 'LOR'}, inplace=True)
X = df.drop('Chance_of_Admit', axis=1)
y = df['Chance_of_Admit']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Train the Ridge Regression model
ridge = Ridge(alpha=5.0)
ridge.fit(X_scaled_df, y)

# Save the model and scaler as pickle files
pickle.dump(ridge, open('model.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))



In [3]:
# Initialize Flask App
app = Flask(__name__)
CORS(app)

@app.route('/')
def home():
    """
    Render the front-end HTML page
    """
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    """
    Handle the prediction API request
    """
    try:
        # Receive input data from frontend
        data = request.get_json()
        gre = float(data['GRE Score'])
        toefl = float(data['TOEFL Score'])
        rating = float(data['University Rating'])
        sop = float(data['SOP'])
        lor = float(data['LOR'])
        cgpa = float(data['CGPA'])
        research = int(data['Research'])

        # Load the model and scaler
        model = pickle.load(open('model.pkl', 'rb'))
        scaler = pickle.load(open('scaler.pkl', 'rb'))

        # Prepare input for prediction
        input_data = np.array([[gre, toefl, rating, sop, lor, cgpa, research]])
        input_data_scaled = scaler.transform(input_data)

        # Make prediction
        prediction = model.predict(input_data_scaled)
        response = {
            'Chance of Admit': round(prediction[0], 4),
            'Model': 'Ridge Regression',
            'Confidence': 'High'
        }

        return jsonify(response)

    except Exception as e:
        # Handle any unexpected error
        return jsonify({'error': str(e)})





In [4]:
@app.route('/health', methods=['GET'])
def health():
    """
    Health check API
    """
    return jsonify({'status': 'running'})




In [6]:
if __name__ == '__main__':
    # Run the Flask app
    app.run(port=5000, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [2]:
df=pd.read_csv('jamboree_admission.csv').drop(columns=['Serial No.'])

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   GRE Score          500 non-null    int64  
 1   TOEFL Score        500 non-null    int64  
 2   University Rating  500 non-null    int64  
 3   SOP                500 non-null    float64
 4   LOR                500 non-null    float64
 5   CGPA               500 non-null    float64
 6   Research           500 non-null    int64  
 7   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 31.4 KB
None
        GRE Score  TOEFL Score  University Rating         SOP       LOR   \
count  500.000000   500.000000         500.000000  500.000000  500.00000   
mean   316.472000   107.192000           3.114000    3.374000    3.48400   
std     11.295148     6.081868           1.143512    0.991004    0.92545   
min    290.000000    92.000000           1.000000    1.0

In [4]:
df.rename(columns={'Chance of Admit ':'Chance_of_Admit','LOR ':'LOR'},inplace=True)

In [5]:
X = df.drop('Chance_of_Admit', axis=1)
y = df['Chance_of_Admit']

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
ridge = Ridge(alpha=5.0)
ridge.fit(X_scaled_df, y)

Ridge(alpha=5.0)

In [8]:
pickle.dump(ridge, open('model.pkl', 'wb'))
pickle.dump(scaler, open('scaler.pkl', 'wb'))

In [9]:
app = Flask(__name__)
CORS(app)

In [10]:
@app.route('/')
def home():
    return render_template('index.html')


In [11]:


@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        gre = data['GRE Score']
        toefl = data['TOEFL Score']
        rating = data['University Rating']
        sop = data['SOP']
        lor = data['LOR']
        cgpa = data['CGPA']
        research = data['Research']

        # Load the model and scaler
        model = pickle.load(open('model.pkl', 'rb'))
        scaler = pickle.load(open('scaler.pkl', 'rb'))

        # Prepare input
        input_data = np.array([[gre, toefl, rating, sop, lor, cgpa, research]])
        input_data_scaled = scaler.transform(input_data)

        # Predict
        prediction = model.predict(input_data_scaled)
        response = {
            'Chance of Admit': round(prediction[0], 4),
            'Model': 'Ridge Regression',
            'Confidence': 'High'
        }
        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)})

In [12]:
@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'running'})



In [14]:
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2025-03-10 20:09:01,304] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "F:\Anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "F:\Anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "F:\Anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "F:\Anaconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "F:\Anaconda3\Lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_func